In [ ]:
import httpx
c = httpx.AsyncClient()

team_id = ""
base_url = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

In [2]:
problems = [
    "probatio",
    "primus",
    "secundus",
    "tertius",
    "quartus",
    "quintus"
]

In [ ]:
import httpx
from pydantic import BaseModel

c = httpx.Client()

team_id = ""
team_id = ""
base_url = "https://31pwr5t6ij.execute-api.eu-west-2.amazonaws.com"

problems = [
    "probatio",
    "primus",
    "secundus",
    "tertius",
    "quartus",
    "quintus"
]

def select(problemName: str):
    return c.post(f"{base_url}/select", json={
        "id": team_id,
        "problemName": problemName
    })

def explore(plans: list[str]) -> list[list[int]]:
    r = c.post(f"{base_url}/explore", json={
        "id": team_id,
        "plans": plans,
    })
    if r.status_code != 200:
        print(r.text)
        r.raise_for_status()
    return (r).json()["results"]

class V(BaseModel):
    room: int
    door: int

def guess(rooms: list[int], startingRoom: int, connections: list[tuple[V,V]]):
    print(connections)
    return c.post(f"{base_url}/guess", json={
        "id": team_id,
        "map": {
            "rooms": rooms,
            "startingRoom": startingRoom,
            "connections": [
                {"from": v1.model_dump_json(), "to": v2.model_dump_json()} for (v1,v2) in connections
            ]
        }
    })


In [20]:
import random
from z3 import *

# --- Step 1: Mock Server for "probatio" (3 rooms) ---
# これは、ローカルテストのためにコンテストサーバーをシミュレートします。
# 実際のコンテストでは、この部分をAPIクライアントに置き換えます。
class MockAedificium:
    """
    ランダムな地図を生成し、探検計画に対する結果を返すモックサーバー。
    """
    def __init__(self, num_rooms=3):
        self.num_rooms = num_rooms
        # 部屋のラベルをランダムに割り当て (0-3)
        self.rooms_label = [random.randint(0, 3) for _ in range(num_rooms)]
        self.starting_room = random.randint(0, num_rooms - 1)
        self.connections = [{} for _ in range(num_rooms)]

        # 全てのドアが繋がるようにランダムな接続を生成
        available_doors = []
        for r in range(num_rooms):
            for d in range(6):
                available_doors.append((r, d))
        
        random.shuffle(available_doors)
        for i in range(0, len(available_doors) - 1, 2):
            r1, d1 = available_doors[i]
            r2, d2 = available_doors[i+1]
            self._add_connection(r1, d1, r2, d2)
            
        print("--- Mock Server Initialized ---")
        print(f"Room Labels: {self.rooms_label}")
        print(f"Starting Room ID: {self.starting_room} (Label: {self.rooms_label[self.starting_room]})")
        print("-----------------------------")

    def _add_connection(self, r1, d1, r2, d2):
        self.connections[r1][d1] = (r2, d2)
        self.connections[r2][d2] = (r1, d1)

    def explore(self, plans):
        """探検計画を実行し、観測されたラベルのシーケンスを返す。"""
        all_results = []
        for plan in plans:
            current_room = self.starting_room
            result_sequence = []
            # 各ステップで、移動先の部屋のラベルを記録する
            for door_char in plan:
                door = int(door_char)
                if door not in self.connections[current_room]:
                    # ありえないが、念のため
                    raise ValueError(f"Room {current_room} has no door {door}")
                
                next_room, _ = self.connections[current_room][door]
                current_room = next_room
                result_sequence.append(self.rooms_label[current_room])
            all_results.append(result_sequence)
        return all_results

    def get_start_label(self):
        return self.rooms_label[self.starting_room]

# --- Step 2: The Z3-based Map Solver ---
# 観測データから地図の制約を構築し、解を見つけるコアロジック。
class MapSolver:
    def __init__(self, num_rooms):
        self.num_rooms = num_rooms
        self.observations = []
        self.start_label = -1

    def add_observation(self, plan, result):
        self.observations.append((plan, result))

    def set_start_label(self, label):
        self.start_label = label

    def solve(self, max_models=2):
        """
        観測データと矛盾しないモデル（地図）を見つけようと試みる。
        見つかったモデルのリストを返す。
        """
        n = self.num_rooms
        if not self.observations:
            print("解くための観測データがありません。")
            return []

        s = Solver()

        # --- 変数定義 ---
        # rooms_label[i]: 部屋iのラベル
        LabelArray = ArraySort(IntSort(), IntSort())
        rooms_label = Const('rooms_label', LabelArray)
        
        # starting_room: 開始部屋のID
        starting_room = Int('starting_room')

        # transition_map[p_id]: 行き先のp_id'。ここで p_id = room_id * 6 + door_id
        TransitionArray = ArraySort(IntSort(), IntSort())
        transition_map = Const('transition_map', TransitionArray)

        # --- 制約定義 ---
        # 1. 基本的なドメイン制約
        i = Int('i')
        s.add(ForAll(i, Implies(And(0 <= i, i < n), 
                                      And(0 <= Select(rooms_label, i), Select(rooms_label, i) <= 3))))
        s.add(0 <= starting_room, starting_room < n)
        s.add(Select(rooms_label, starting_room) == self.start_label)

        # 2. 接続の制約（無向グラフ）
        # transition_mapは対合(involution)である。つまり、f(f(x)) = x。
        # (部屋A,ドアX)が(部屋B,ドアY)に繋がるなら、(部屋B,ドアY)は(部屋A,ドアX)に繋がる。
        p = Int('p')
        s.add(ForAll(p, Implies(And(0 <= p, p < n * 6), 
                                And(0 <= Select(transition_map, p), Select(transition_map, p) < n * 6))))
        s.add(ForAll(p, Implies(And(0 <= p, p < n * 6), Select(transition_map, Select(transition_map, p)) == p)))
        # どのドアも自分自身には繋がらない
        s.add(ForAll(p, Implies(And(0 <= p, p < n * 6), Select(transition_map, p) != p)))

        # 3. 観測データとの整合性
        for obs_idx, (plan, result) in enumerate(self.observations):
            path = [Int(f'path_{obs_idx}_{t}') for t in range(len(plan) + 1)]
            s.add(path[0] == starting_room)

            for t, door_char in enumerate(plan):
                # 遷移の制約: 現在の部屋と通るドアから、次の部屋が決まる
                door = int(door_char)
                current_room = path[t]
                p_id = current_room * 6 + door
                next_p_id = Select(transition_map, p_id)
                next_room = next_p_id / 6
                s.add(path[t+1] == next_room)

                # ラベルの制約: 到着した部屋のラベルが観測結果と一致する
                s.add(Select(rooms_label, path[t+1]) == result[t])
        
        # --- モデル（解）の探索 ---
        found_models = []
        while len(found_models) < max_models:
            if s.check() == sat:
                m = s.model()
                found_models.append(m)
                
                # このモデルを無効化する制約を追加し、次の解を探す
                # ここでは単純化のため、transition_mapが同じにならないようにする
                or_clause = []
                for i in range(n * 6):
                    val = m.eval(Select(transition_map, i))
                    or_clause.append(Select(transition_map, i) != val)
                s.add(Or(or_clause))
            else:
                break # これ以上モデルは見つからない

        return found_models

    def model_to_map_dict(self, model):
        """Z3モデルを、/guess APIで要求される辞書形式に変換する。"""
        n = self.num_rooms
        
        # z3の配列オブジェクトを取得
        label_array_obj = model.eval(Const('rooms_label', ArraySort(IntSort(), IntSort())))
        transition_array_obj = model.eval(Const('transition_map', ArraySort(IntSort(), IntSort())))

        # ラベルを評価
        labels = [model.eval(Select(label_array_obj, i)).as_long() for i in range(n)]
        
        # 開始部屋を評価
        start_room_id = model.eval(Int('starting_room')).as_long()

        # 接続を評価
        connections = []
        added_connections = set()

        for from_room in range(n):
            for from_door in range(6):
                p_id = from_room * 6 + from_door
                if (from_room, from_door) in added_connections:
                    continue
                
                to_p_id = model.eval(Select(transition_array_obj, p_id)).as_long()
                to_room = to_p_id // 6
                to_door = to_p_id % 6
                
                connections.append({
                    "from": {"room": from_room, "door": from_door},
                    "to": {"room": to_room, "door": to_door}
                })
                added_connections.add((from_room, from_door))
                added_connections.add((to_room, to_door))

        return {
            "rooms": labels,
            "startingRoom": start_room_id,
            "connections": connections
        }

NUM_ROOMS = 3

select(problems[0])
# 1. 環境のセットアップ
solver = MapSolver(num_rooms=NUM_ROOMS)

# 2. 初期探検
# 十分な情報を集めるために、ある程度の長さのランダムな経路をいくつか試す
initial_plans = ["".join([str(random.randint(0,5)) for _ in range(NUM_ROOMS * 18)])]

print(f"実行する初期探検計画: {initial_plans}")
results = explore(initial_plans)
print(f"観測結果: {results}")

# 3. 観測データをソルバーに追加
solver.set_start_label("0")
for plan, result in zip(initial_plans, results):
    solver.add_observation(plan, result)

# 4. z3ソルバーを実行して解を求める
print("\n制約を解いています...")
models = solver.solve(max_models=1)

# 5. 結果の評価
if not models:
    print("解が見つかりませんでした。情報が不足しているか、矛盾があります。")
else:
    print(f"{len(models)}個の整合性のある地図が見つかりました。")
    
    # 最初の解を取得して表示
    first_solution = solver.model_to_map_dict(models[0])
    print("\n--- 最初の解（候補となる地図） ---")
    print(f"部屋のラベル: {first_solution['rooms']}")
    print(f"開始部屋ID: {first_solution['startingRoom']}")
    print("接続:")
    for conn in first_solution['connections']:
        f = conn['from']
        t = conn['to']
        print(f"  (部屋 {f['room']}, ドア {f['door']}) <--> (部屋 {t['room']}, ドア {t['door']})")
    
    if len(models) > 1:
        print("\n注意: 地図を一意に特定できませんでした。追加の探検が必要です。")
    else:
        print("\n地図が一意に特定できました！これを提出します。")
        # ここで /guess API を呼び出す

実行する初期探検計画: ['514224200123033513344533105015411134233300150454323133']
観測結果: [[0, 0, 1, 1, 0, 1, 1, 0, 2, 2, 1, 0, 0, 2, 2, 2, 0, 1, 2, 2, 1, 1, 2, 2, 2, 1, 0, 0, 2, 1, 2, 1, 1, 1, 1, 2, 1, 0, 0, 0, 0, 2, 2, 1, 2, 2, 1, 2, 1, 2, 0, 0, 1, 2, 2]]

制約を解いています...
解が見つかりませんでした。情報が不足しているか、矛盾があります。


In [ ]:
# Action	0	1	2	3	4	5
# State						
# 0	{2, 3}	{0}	{0, 1}	{0, 1}	{0, 1}	{1, 2}
# 1	{1, 3}	{0, 1}	{0, 3}	{0, 1}	{1, 3}	{0, 1}
# 2	{0}	{2}	{2}	{0}	{3}	{3}
# 3	{1}	{2}	{1}	{1}	{2}	{0}

import z3


In [567]:
import random


select(problems[1])
n=6
plans = "".join([str(random.randint(0,5)) for _ in range(n * 18)])
results = "".join([str(a) for a in explore([plans])[0]])
print(plans)
print(results)
length = 6
# 長さが５のsubstrings
substrings = set()
for i in range(len(results) - length + 1):
    substrings.add(results[i:i+length])
print(substrings, len(substrings))

532540305153144121345210104050023351100544244022330455503055315340523010014400004024051444204302255250001212
0132221320021232032131312122332100010121313222101310000000022313121323131231312100012131222122310133200000323
{'121313', '122212', '101332', '212231', '100010', '131210', '232032', '131231', '032131', '213131', '320321', '310133', '210131', '000223', '000000', '131312', '223321', '000022', '123131', '313121', '101310', '210001', '121223', '132313', '022313', '213200', '203213', '132002', '000002', '021232', '310000', '213231', '000101', '131213', '010121', '213122', '222101', '221320', '121312', '121323', '002123', '122310', '101213', '213132', '321000', '332100', '013222', '312313', '313123', '221013', '222122', '320000', '312132', '231013', '200000', '122332', '012131', '313222', '100012', '332000', '121000', '200212', '013100', '322213', '233210', '312122', '320021', '221223', '321313', '212320', '231312', '131222', '013320', '100000', '323131', '312221', '002231', '001213', 

In [171]:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX != rootY:
            if self.rank[rootX] > self.rank[rootY]:
                self.parent[rootY] = rootX
            elif self.rank[rootX] < self.rank[rootY]:
                self.parent[rootX] = rootY
            else:
                self.parent[rootY] = rootX
                self.rank[rootX] += 1
    def roots(self):
        return set(self.find(x) for x in range(len(self.parent)))
def enumerate_string_of_length(n, alphabet):
    if n == 0:
        yield ""
    else:
        for s in enumerate_string_of_length(n - 1, alphabet):
            for letter in alphabet:
                yield s + letter

In [553]:
select(problems[1])
n = 6

result = explore(["1111111111"])[0]
print(result)

[0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]


In [374]:
select(problems[1])
n = 6

sig_len = 5
sig = random.choice(list(enumerate_string_of_length(sig_len, "012345")))
sig = "01201"
print("sig", sig)

cur = sig
while len(cur) < n * 18:
    random_string = "".join([random.choice("012345") for _ in range(2)])
    cur += random_string + sig
if len(cur) > n * 18:
    cur = cur[:n * 18]
print(cur)

result = explore([cur])[0]
print(result)

sig_results = []
sig_positions = [i for i in range(len(cur)) if cur[i:i+sig_len] == sig]

print("sig_positions", len(sig_positions), sig_positions)
for sig_pos in sig_positions:
    sig_result = "".join([str(a) for a in result[sig_pos: sig_pos + sig_len]])
    sig_results.append(sig_result)

print(set(sig_results))
print(len(set(sig_results)))

sig 01201
012015001201540120121012013201201550120122012013201201210120104012010501201250120152012014101201050120135012
[0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 0, 1, 0, 1, 2, 0, 1, 2, 1, 1, 2, 1, 1, 2, 3, 0, 0, 0, 1, 0, 1, 0, 2, 0, 1, 2, 0, 1, 2, 1, 1, 2, 1, 1, 2, 3, 0, 0, 0, 1, 0, 1, 2, 1, 1, 2, 1, 1, 2, 0, 1, 0, 1, 2, 0, 1, 1, 1, 1, 2, 1, 1, 2, 1, 0, 1, 2, 1, 1, 2, 1, 0, 1, 2, 1, 1, 2, 0, 1, 1, 2, 1, 1, 2, 0, 2, 0, 1, 2, 0, 1, 3, 3, 3, 1, 2]
sig_positions 15 [0, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98]
{'11211', '01211', '10120', '20120', '00010'}
5


In [187]:
uf = UnionFind(len(results) - 1)
ss_len = 3
ss_t = list(enumerate_string_of_length(ss_len, "012345"))
for s in ss_t:
    positions = [i for i in range(len(results) - ss_len) if results[i:i+ss_len] == s]
    if len(positions) >= 2:
        corresponding_results = [results[i: i+ss_len] for i in positions]
        if len(set(corresponding_results)) == 1:
            for p1_start, p2_start in zip(positions, positions[1:]):
                for i in range(ss_len):
                    uf.union(p1_start + i, p2_start + i)
print("done")
print(len(uf.roots()))

done
4


In [6]:

def generate_random_graph_parameter_free(room_labels: list[int]):
    """
    パラメータを指定することなく、3頂点の完全にランダムなグラフ情報を生成します。
    自己ループ（部屋単位・ドア単位）も自然な確率で発生します。

    Args:
        room_labels (list[int]): 3つの部屋のラベルのリスト。

    Returns:
        tuple[list[int], int, list[tuple[V, V]]]: 
            - rooms (list[int]): 部屋のラベルリスト
            - startingRoom (int): ランダムな開始部屋のインデックス
            - connections (list[tuple[V,V]]): ランダムな接続情報のリスト
    """
    if len(room_labels) != 3:
        raise ValueError("room_labelsには3つの部屋のラベルを指定してください。")

    # 3部屋 x 6ドア = 18個のすべてのドアをリストアップ
    # (部屋インデックス, ドア番号) のタプルで管理
    available_doors = []
    for room_idx in range(3):
        for door_idx in range(6):
            available_doors.append((room_idx, door_idx))

    connections = []
    
    # 利用可能なドアがなくなるまでペアを作り続ける
    while available_doors:
        # 1. 接続元となるドアd1をランダムに選び、リストから削除する
        d1_index = random.randrange(len(available_doors))
        d1 = available_doors.pop(d1_index)
        
        # 2. 接続先d2の候補リストを作成する
        #    候補は「残りの利用可能なドア」+「d1自身」
        potential_partners = available_doors + [d1]
        
        # 3. 候補の中からランダムにd2を選ぶ
        d2 = random.choice(potential_partners)
        
        # 4. d2がd1自身でなかった場合（自己ループでない場合）、
        #    d2も利用可能リストから削除する
        if d1 != d2:
            available_doors.remove(d2)
            
        # 5. Vモデルのインスタンスを作成して、接続リストに追加する
        v1 = V(room=d1[0], door=d1[1])
        v2 = V(room=d2[0], door=d2[1])
        connections.append((v1, v2))

    # 開始部屋をランダムに選択
    starting_room_index = 0

    return room_labels, starting_room_index, connections


class AedificiumSimulator:
    """
    生成された地図情報をもとに、ルートプランの実行結果をシミュレートするクラス。
    """
    def __init__(self, rooms: list[int], startingRoom: int, connections: list[tuple[V, V]]):
        """
        シミュレータを初期化します。

        Args:
            rooms (List[int]): 各部屋のラベルのリスト。インデックスが部屋IDとなる。
            startingRoom (int): 開始部屋のインデックス (ID)。
            connections (List[Tuple[V, V]]): 部屋の接続情報のリスト。
        """
        self.rooms = rooms
        self.starting_room_index = startingRoom
        
        # 高速に移動先を検索するための「遷移マップ」を構築する
        # (部屋ID, ドア番号) -> (次の部屋ID, 入るドア番号)
        self.transition_map: dict[tuple[int, int], tuple[int, int]] = {}
        self._build_transition_map(connections)

    def _build_transition_map(self, connections: list[tuple[V, V]]):
        """
        接続リストから、双方向の遷移マップを効率的に構築します。
        """
        for v1, v2 in connections:
            # from v1 to v2
            from_key = (v1.room, v1.door)
            to_value = (v2.room, v2.door)
            self.transition_map[from_key] = to_value

            # from v2 to v1 (グラフは無向なので、逆方向の接続も追加)
            from_key_reverse = (v2.room, v2.door)
            to_value_reverse = (v1.room, v1.door)
            self.transition_map[from_key_reverse] = to_value_reverse
        
        # 接続されていないドアがないかチェック (デバッグ用)
        total_doors = len(self.rooms) * 6
        if len(self.transition_map) != total_doors:
            print(f"警告: 遷移マップのエントリ数が{len(self.transition_map)}です。"
                  f"想定されるドアの総数({total_doors})と一致しません。")

    def simulate_plan(self, plan: str) -> list[int]:
        """
        与えられたルートプランを実行し、観測される部屋ラベルのリスト（アドソの記録）を返します。

        Args:
            plan (str): '0'から'5'の数字で構成されるルートプラン文字列。例: "0325"

        Returns:
            List[int]: ルートプランに従って移動した際に観測される部屋ラベルのシーケンス。
        """
        if not plan:
            return []

        current_room_index = self.starting_room_index
        observed_labels = [current_room_index]

        for door_char in plan:
            exit_door = int(door_char)
            
            # 現在の部屋と出るドアから、次の部屋と入るドアを検索
            lookup_key = (current_room_index, exit_door)
            destination = self.transition_map.get(lookup_key)

            if destination is None:
                raise ValueError(f"地図が不正です: 部屋{current_room_index}のドア{exit_door}からの接続が見つかりません。")

            next_room_index, _ = destination # 入るドアの情報は今回は使わない
            
            # 次の部屋に移動
            current_room_index = next_room_index
            
            # 移動先の部屋のラベルを記録
            observed_labels.append(self.rooms[current_room_index])
            
        return observed_labels


attempts = 0
while True:
    attempts += 1
    print(f"--- Attempt #{attempts} ---")
    await select(problems[0])
    plans = "".join([str(random.randint(0,5)) for _ in range(100)])
    results = await explore(plans)

    cou = 0
    while True:
        cou += 1
        # ランダムなグラフを生成
        rooms, startingRoom, connections = generate_random_graph_parameter_free([0,1,2])
        sim = AedificiumSimulator(rooms=rooms, startingRoom=startingRoom, connections=connections)
        r = sim.simulate_plan(plans)
        if cou % 10000 == 0:
            print(cou)
            break
        if r == results:
            print("fount")
            r =await guess(
                rooms=[0,1,2], 
                startingRoom=0,
                connections=connections
            )
            response_json = r.json()
            print(f"Response: {response_json}")
            if response_json.get("correct"):
                print("正解しました！")
                break
            else:
                print("不正解です。新しい問題で再挑戦します。")
                # 不正解の場合、問題がリセットされるので再度selectから始める必要がある
                await select(problems[0])
    # print(f"Guessing with startingRoom={startingRoom}")

    # # guessを試行
    # # 注: あなたのguess関数のURLが/exploreになっていたので/guessに修正しています
    # r = await c.post(f"{base_url}/guess", json={
    #     "id": team_id,
    #     "map": {
    #         "rooms": rooms,
    #         "startingRoom": startingRoom,
    #         "connections": [
    #             # Vモデルを辞書に変換する必要がある
    #             {"from": v1.model_dump(), "to": v2.model_dump()} for (v1,v2) in connections
    #         ]
    #     }
    # })
    
    # response_json = r.json()
    # print(f"Response: {response_json}")

    # if response_json.get("correct"):
    #     print("正解しました！")
    #     break
    # else:
    #     print("不正解です。新しい問題で再挑戦します。")
    #     # 不正解の場合、問題がリセットされるので再度selectから始める必要がある
    #     await select(problems[0])

--- Attempt #1 ---
2044545554303543341201520544244101225501251312154352154232153440100500050123013141221533413442550504
{'error': 'Error: For input string: ""'}


KeyError: 'results'

In [ ]:
8 / 30

0.26666666666666666

In [ ]:
await guess(
    rooms
)